# Лабораторная работа 2. Методы анализа параметров компонентов моделей функционирования КИС

## Вариант 2

###### Выполнил: Борисочкин М. И. РТ5-81Б

### Задание 1

In [99]:
# Задание значений случайных величин и их вероятностей
Variant <- 2
set.seed(Variant)

X1 <- sample(c(1:20), 5)
X2 <- sample(c(3:100), 5)
X3 <- sample(c(0:40), 5)

pp1 <- runif(5)
p1 <- pp1/sum(pp1)
pp2 <- runif(5)
p2 <- pp2/sum(pp2)
pp3 <- runif(5)
p3 <- pp3/sum(pp3)

In [100]:
# Вывод значений
Data <- data.frame(X1, X2, X3, p1, p2, p3)
View(Data)

X1,X2,X3,p1,p2,p3
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
15,95,10,0.03550468,0.15066048,0.066636399
6,83,0,0.31342599,0.21204922,0.005242698
19,78,2,0.18351462,0.06474921,0.082881318
8,43,15,0.39628843,0.15490792,0.407852759
1,52,31,0.07126628,0.41763318,0.437386826


#### Численный метод

In [101]:
Data1 <- data.frame(Data$X1, Data$p1)
colnames(Data1) <- c("X", "p")
Data2 <- data.frame(Data$X2, Data$p2)
colnames(Data2) <- c("X", "p")
Data3 <- data.frame(Data$X3, Data$p3)
colnames(Data3) <- c("X", "p")

In [102]:
# Функция для генерации n значений с заданным распределением вероятности
genseries <- function(data, n) {
  res <- c()
  for(i in c(1:n))
  {
    punif <- runif(1)
    if(punif <= data$p[1]) res[i] <- data$X[1]

    if(punif > data$p[1] & punif <= data$p[1] + data$p[2]) res[i] <- data$X[2]

    if(punif > data$p[1] + data$p[2] & punif <= data$p[1] + data$p[2] + data$p[3])
    {
      res[i] <- data$X[3]
    } 

    if( punif > data$p[1] + data$p[2] + data$p[3] & punif <= data$p[1] + data$p[2] + data$p[3] + data$p[4])
    {
      res[i] <- data$X[4]
    }

    if(punif > data$p[1] + data$p[2] + data$p[3] + data$p[4] & punif <= 1) {
      res[i] <- data$X[5]
    }
  }
  return(res)
}

In [103]:
# Создание нового датафрейма
X1c <- genseries(Data1, 2000000)
X2c <- genseries(Data2, 2000000)
X3c <- genseries(Data3, 2000000)

Data123 <- data.frame(X1c, X2c, X3c, X1c + X2c + X3c)
colnames(Data123) <- c("X1", "X2", "X3", "X1 + X2 + X3")

In [104]:
# Мат. ожидание численно
mean(Data123$`X1 + X2 + X3`)

[1] 95.00512

In [105]:
# Дисперсия численно
var(Data123$`X1 + X2 + X3`)

[1] 486.2732

In [106]:
# Среднеквадратичное отклонение численно
sd(Data123$`X1 + X2 + X3`)

[1] 22.0516

In [107]:
# Коэффициент вариации численно
sd(Data123$`X1 + X2 + X3`)*100/mean(Data123$`X1 + X2 + X3`)

[1] 23.21096

#### Теоритический метод (по определению)

In [108]:
# Матиматическое ожидание (по определению)
MX <- function(X, p) return(sum(X*p)) 

In [109]:
# Матожидание случайной величины X1 + X2 + X3
MX1 <- MX(Data$X1, Data$p1)
MX2 <- MX(Data$X2, Data$p2)
MX3 <- MX(Data$X3, Data$p3)

MX123def <- MX1 + MX2 + MX3

MX123def

[1] 94.99162

In [110]:
# Дисперсия (по определению)
DX <- function(X, p) return(sum(p*(X - MX(X, p))^2))

In [111]:
# Дисперсия случайной величины X1 + X2 + X3
DX123def <- DX(Data$X1, Data$p1) + DX(Data$X2, Data$p2) + DX(Data$X3, Data$p3)
DX123def

[1] 486.5432

In [112]:
# Среднеквадратичное отклонение случайной величины X1 + X2 + X3
std123def <- sqrt(DX123def)
std123def

[1] 22.05772

In [113]:
# Коэффициент вариации случайной величины X1 + X2 + X3
varcoef123def <- std123def*100/MX123def
varcoef123def

[1] 23.2207

#### Теоритический метод (с использованием производящей функции)

Производящие функции:

$P_{X_1}(z) = 0.07126628z + 0.31342599z^6 + 0.39628843z^8 + 0.03550468z^{15} + 0.18351462z^{19}
\\ P_{X_2}(z) = 0.15490792z^{43} + 0.41763318z^{52} + 0.06474921z^{78} + 0.21204922z^{83} + 0.15066048z^{95}
\\ P_{X_3}(z) = 0.005242698z^0 + 0.082881318z^2 + 0.066636399z^{10} + 0.407852759z^{15} + 0.437386826z^{31} $

In [114]:
# Производящая функция случайной величины X1 + X2 + X3
PX123 <- expression((0.07126628*z + 0.31342599*z^6 + 0.39628843*z^8 + 0.03550468*z^15 + 0.18351462*z^19)*
  (0.15490792*z^43 + 0.41763318*z^52 + 0.06474921*z^78 + 0.21204922*z^83 + 0.15066048*z^95)*
  (0.005242698*z^0 + 0.082881318*z^2 + 0.066636399*z^10 + 0.407852759*z^15 + 0.437386826*z^31))
PX123

expression((0.07126628 * z + 0.31342599 * z^6 + 0.39628843 * 
    z^8 + 0.03550468 * z^15 + 0.18351462 * z^19) * (0.15490792 * 
    z^43 + 0.41763318 * z^52 + 0.06474921 * z^78 + 0.21204922 * 
    z^83 + 0.15066048 * z^95) * (0.005242698 * z^0 + 0.082881318 * 
    z^2 + 0.066636399 * z^10 + 0.407852759 * z^15 + 0.437386826 * 
    z^31))

In [115]:
# Матожидание случайной величины X1 + X2 + X3 (по св-ву 2 производящей функции)
PX123d <- D(PX123, 'z') # Первая производная от производящей функции
MX123pf <- eval(PX123d, z <- 1)
MX123pf

[1] 94.99162

In [116]:
# Дисперсия случайной величины X1 + X2 + X3 (по св-ву 3 производящей функции)
PX123dd <- D(PX123d, 'z') # Вторая производная от производящей функции
DX123pf <- eval(PX123dd, z <- 1) + MX123pf - MX123pf^2
DX123pf

[1] 486.5431

In [117]:
# Среднеквадратичное отклонение случайной величины X1 + X2 + X3
std123pf <- sqrt(DX123pf)
std123pf

[1] 22.05772

In [118]:
# Коэффициент вариации случайной величины X1 + X2 + X3
varcoef123pf <- std123pf*100/MX123pf
varcoef123pf

[1] 23.2207

### Задание 2

In [119]:
Variant <- 2
set.seed(Variant) 
runif(3)

[1] 0.1848823 0.7023740 0.5733263

$\lambda_1 = 0.18488225992769;
\\ λ_2 = 0.702374035958201;
\\ λ_3 = 0.573326334822923.$

В качестве значения параметра $τ$ возьмём номер варианта, т. е. 2:

$ τ = 2.$

In [120]:
# Параметры распределения для X1, X2, X3
lamda1 = 0.18488225992769
lamda2 = 0.702374035958201
lamda3 = 0.573326334822923
tau = 2

#### Численный метод

In [121]:
# Генерация значений случайной величины X1 + X2 + X3
X1 <- rpois(5000000, lamda1*tau)
X2 <- rpois(5000000, lamda2*tau)
X3 <- rpois(5000000, lamda3*tau)

Data123pois <- data.frame(X1, X2, X3, X1 + X2 + X3)
colnames(Data123pois) <- c("X1", "X2", "X3", "X1 + X2 + X3")

In [122]:
# Мат. ожидание численно
mean(Data123pois$`X1 + X2 + X3`)

[1] 2.921279

In [123]:
# Дисперсия численно
var(Data123pois$`X1 + X2 + X3`)

[1] 2.923657

In [124]:
# Среднеквадратичное отклонение численно
sd(Data123pois$`X1 + X2 + X3`)

[1] 1.70987

In [125]:
# Коэффициент вариации численно
sd(Data123pois$`X1 + X2 + X3`)*100/mean(Data123pois$`X1 + X2 + X3`)

[1] 58.53156

#### Теоритический метод

Производящая функция случайной величины, распределённой по закону Пуассона:

$P_X(z) = ∑_{n = 0}^{∞}p_nz^n = ∑_{n = 0}^{∞}\frac{(λt)^n}{n!}e^{-λt}z^n = e^{-λt}∑_{n = 0}^{∞}\frac{(λtz)^n}{n!} = e^{-λt} \cdot e^{λtz} = e^{-λt(1 - z)}$

Тогда, применяя св-во 5 производящей функции из лекции, производящая функция случайной величины $X_1 + X_2 + X_3$ равно:

$P_{X_1 + X_2 + X_3}(z) = P_{X_1}(z) \cdot P_{X_2}(z) \cdot P_{X_3}(z) = e^{-λ_1t(1-z)} \cdot e^{-λ_2t(1-z)} \cdot e^{-λ_3t(1-z)} = e^{-λ_1t(1-z) + (-λ_2t(1-z)) + (-λ_3t(1-z))} = e^{-(λ_1 + λ_2 + λ_3) \cdot t \cdot (1-z)}$

Значения математического ожидания и дисперсии $X_1 + X_2 + X_3$ по св-ву 2 и 3, соответственно:

$M(X_1 + X_2 + X_3) = P_{X_1 + X_2 + X_3}^{'}(z)|_{z = 1} = (λ_1 + λ_2 + λ_3)t \cdot e^{-(λ_1 + λ_2 + λ_3) \cdot t \cdot (1-z)}|_{z = 1} = (λ_1 + λ_2 + λ_3)t$

$D(X_1 + X_2 + X_3) = P_{X_1 + X_2 + X_3}^{''}(z)|_{z = 1} + P_{X_1 + X_2 + X_3}^{'}(z)|_{z = 1} - (P_{X_1 + X_2 + X_3}^{'}(z))^2|_{z = 1} = ((λ_1 + λ_2 + λ_3)t)^2 + (λ_1 + λ_2 + λ_3)t - ((λ_1 + λ_2 + λ_3)t)^2 = (λ_1 + λ_2 + λ_3)t$


In [126]:
# Матожидание и дисперсия случайной величины X1 + X2 + X3
MDX123 <- (lamda1 + lamda2 + lamda3)*tau
MDX123

[1] 2.921165

In [127]:
# Среднеквадратичное отклонение случайной величины X1 + X2 + X3
std123 <- sqrt(MDX123)
std123

[1] 1.709142

In [128]:
# Коэффициент вариации случайной величины X1 + X2 + X3
# Коэффициент вариации случайной величины X1 + X2 + X3
varcoef123 <- std123*100/MDX123
varcoef123

[1] 58.5089

### Задание 3

In [129]:
# Значение параметра эксподенциального распределения
Variant <- 2
set.seed(Variant) 
lambda<-runif(7)[7]
lambda

[1] 0.129159

Для теоритических расчётов и для эксперимента будем считать, что у нас 7 последовательных программ, т. е. k = 7.

In [130]:
# Число последовательных программ
k <- 7

#### Теоритическая часть

Преобразование Лапласа-Стилтьеса для длительности работы i-ой программы:

$Β_{τ_i}(s) = ∫_{0}^{∞} e^{-st} dF(t) = ∫_{0}^{∞} e^{-st} \cdot f(t) dt = ∫_{0}^{∞} e^{-st} \cdot λe^{-λt} dt =∫_{0}^{∞} λe^{(-s - λ)t}dt = - \frac{λ e^{(-s - λ)t}}{λ + s}|_{0}^{∞} = 0 - (-\frac{λ}{λ + s}) = \frac{λ}{λ + s}$

Тогда по свойству 9 из лекции 2 преобразование Лапласа-Стилтьеса для всей ВС выглядит следующим образом:

$Β_{τ}(s) = Β_{τ_1}(s) \cdot Β_{τ_2}(s) ⋅ ... ⋅Β_{τ_k}(s) = (Β_{τ_i}(s))^k = (\frac{λ}{λ + s})^k$

Математическое ожиданние по св-ву 8 из лекции равно:

$M(τ) = -Β_{τ}(s)^{'}|_{s = 0} = -(-\frac{k \cdot λ^k}{(λ + s)^{k + 1}})|_{s = 0} = \frac{k \cdot λ^k}{(λ + s)^{k + 1}}|_{s = 0} = \frac{k \cdot λ^k}{λ^{k + 1}} = \frac{k}{λ}$

Дисперсия по св-ву 8 из лекции равна:

$D(τ) = Β_{τ}(s)^{''}|_{s = 0}  - (Β_{τ}(s)^{'})^2|_{s = 0} = \frac{k \cdot λ^k ⋅(k + 1)}{(λ + s)^{k + 2}}|_{s = 0} - (\frac{k}{λ})^2 = \frac{k \cdot λ^k ⋅(k + 1)}{λ^{k + 2}} - \frac{k^2}{λ ^2} =  \frac{k^2 + k}{λ^{2}} - \frac{k^2}{λ ^2} = \frac{k}{λ ^2}$

In [131]:
# Математическое ожидание времени обработки сообщения
MXtau <- expression(k/lambda)
eval(MXtau)

[1] 54.19677

In [132]:
# Диссперсия времени обработки сообщения
DXtau <- expression(k/lambda^2)
eval(DXtau)

[1] 419.6129

#### Экспериментальная часть

In [133]:
# Генерация периодов обработки сообщения последовательными программами
Variant <- 2
set.seed(Variant) 

expData <- data.frame()
for(i in c(1:k)) {
  x <- rexp(250000, lambda)
  expData <- rbind(expData, x)
}
expData <- t(expData)
expData <- data.frame(expData, rowSums(expData))
rownames(expData) <- c(1:250000)
colnames(expData) <- c("X1", "X2", "X3", "X4", "X5", "X6", "X7", "Summ")

In [134]:
# Математическое ожидание времени обработки сообщения
mean(expData$Summ)

[1] 54.1331

In [135]:
# Диссперсия времени обработки сообщения
var(expData$Summ)

[1] 419.0736

### Задание 4

Доказать, что сумма величин, распределенных по закону Пуассона с параметрами $α_1,α_2,…$ , распределена по закону Пуассона, также обосновать это экспериментально.

Доказательство:

Для доказательста будем использовать следующее утверждение из лекции:
> Мы получили, что дисперсия случайной величины, распределенной по закону Пуассона, равна ее математическому ожиданию. Это свойство можно использовать для проверки гипотезы о том, что заданная случайная величина имеет распределение Пуассона - если на экспериментальных данных оценки дисперсии и мат.ожидания примерно равны, то это указывает в пользу закона распределения Пуассона.

Математическое ожидание и дисперсия случайной величины по закону Пуассона имеют вид:

$ MX = DX = λ.$

Представим данные нам случайные величины с помощью производящих функций:

$P_{X_i} = ∑_{n = 0}^∞\frac{(α_i \cdot τ)^n}{n!}e^{-α_i ⋅ τ}⋅z^n = e^{-a_it(z-1)}$ (см. задачу 2)

Тогда производящая функция суммы этих величин будет равна следующему (по св-ву):

$
P_{Σ} = \Pi_{i = 1}^∞P_{X_i} = \Pi_{i = 1}^∞e^{-a_it(z-1)} = e^{-at(z-1)},
$
где $α = ∑_{i = 1}^{∞}α_i$

Математическое ожидание по св-ву производящей функции равно следующему:

$M(ΣX) = P_{Σ}^{'}(z)|_{z = 1} = α⋅t$ (см. задачу 2)

Дисперсия, в свою очередь, равна следующему (также по св-ву производящей функции):

$D(ΣX) = P_{Σ}^{''}(z)|_{z = 1} + P_{Σ}^{'}(z)|_{z = 1} - (P_{Σ}^{'}(z))^2|_{z = 1} = α⋅t$ (см. задачу 2)

Итого мы получили следующее:

$M(ΣX) = α⋅t = D(ΣX)$

Математическое ожидание суммы величин, распределенных по закону Пуассона, равно дисперсии той же суммы. Из чего можно сделать вывод, что данная сумма тоже распределена по закону Пуассона.

Обоснуем наше доказательство экпериментально: проверим данное утверждение для 10 случайных величин распределённый по Пуассону.

In [152]:
# Генерация 10 расперделённых по Пуассону величин, их суммы, и величины распределённой по Пуассону с параметром равным сумме первых десяти
Variant <- 2
set.seed(Variant)

lambdas <- c(runif(10))
tau <- 10
poisData <- data.frame()

for (i in c(1:10))
{
  X <- rpois(500000, lambdas[i]*tau)
  poisData <- rbind(poisData, X)
}

poisData <- t(poisData)
poisData <- data.frame(poisData, rowSums(poisData))
poisData <- data.frame(poisData, rpois(500000, sum(lambdas)*tau))

rownames(poisData) <- c(1:500000)
colnames(poisData) <- c("X1", "X2", "X3", "X4", "X5", "X6", "X7"," X8",
                       "X9", "X10", "Summ", "Sigma")

In [153]:
# Матожидание суммы 10 величин, распределённых по закону Пуассона
mean(poisData$Summ)

[1] 54.96904

In [154]:
# Дисперсия суммы 10 величин, распределённых по закону Пуассона
var(poisData$Summ)

[1] 54.93976

In [155]:
# Матожидание величины, распределённой по закону Пуассона, с параметром равной сумме параметров первых десяти распределений
mean(poisData$Sigma)

[1] 54.93387

In [156]:
# Дисперсия величины, распределённой по закону Пуассона, с параметром равной сумме параметров первых десяти распределений
var(poisData$Sigma)

[1] 54.66208

### Задание 5

In [159]:
# Значение параметра эксподенциального распределения
Variant<-2 
set.seed(Variant)
lambda<-runif(100)[77]
lambda

[1] 0.2880597

Найдём функцию плотности вероятности времени безотказной работы всей системы с помощью свойства 9 преобразования Лапласа-Стилтьеса:

$f_t = ∫_{A(t)}^{B(t)}f_{t_1}(t_1) \cdot f_t2{t - t_1} dt_1 = ∫_{A(t)}^{B(t)} λe^{-λt_1}\cdot λe^{-λ(t - t_1)} dt_1 = λ^2 \cdot ∫_{A(t)}^{B(t)} e^{-λt_1}\cdot e^{-λt} \cdot e^{λt_1} dt_1 = λ^2 \cdot ∫_{A(t)}^{B(t)} e^{-λt}dt_1$ 

Теперь посчитаем пределы интегрирования:

$0 < t_1 < ∞;
\\ 0 < t_2 < ∞;
\\ 0 < t < ∞.
\\\
\\a = 0, b = ∞, c = 0, d = ∞.
\\\
\\ A(t) = a + \frac{t - (a + d) + |t - (a + d)|}{2} = a + \frac{t - (a + d) + (a + d) - t}{2} = a = 0;
\\ B(t) = b + \frac{t - (b + c) - |t - (b + c)|}{2} = b + \frac{t - (b + c) - (b + c) + t}{2} = b + t - b - c = t$

Тогда функция плотнотсти равна:

$f_t = λ^2 \cdot ∫_{A(t)}^{B(t)} e^{-λ(t_1 + t_2)}dt_1 = λ^2 \cdot ∫_{0}^{t} e^{-λt}dt_1 = λ^2 \cdot e^{-λt}t_1|_0^t =  λ^2 \cdot e^{-λt}t$

Теперь для поиска математического ожидания необходимо определить преобразования Лапласа-Стилтьеса для найденной функции:

$Β_{t}(s) = ∫_{0}^{∞} e^{-st} dF(t) = ∫_{0}^{∞} e^{-st} \cdot f(t) dt = ∫_{0}^{∞} e^{-st} \cdot λ^2 \cdot e^{-λt}t dt = λ^2 \cdot∫_{0}^{∞} e^{-st} \cdot e^{-λt} \cdot t dt = λ^2 \cdot∫_{0}^{∞} e^{-(s + λ)t} \cdot t dt =λ^2 \cdot( - \frac{(s+ λ)t + 1}{(s + λ)^2} ⋅ e^{-(s + λ)t})|_0^∞ = λ^2 \cdot (0 - (-\frac{1}{(s + λ)^2} ⋅ 1)) = \frac{λ^2}{(s + λ)^2}$

Тогда математическое ожидание равно:

$M(t) = -Β_{t}(s)^{'}|_{s = 0} = -(-\frac{2λ^2 \cdot}{(s + λ)^3})|_{s = 0} = \frac{2}{λ}$

Дисперсия в свою очередь равна:
$D(τ) = Β_{τ}(s)^{''}|_{s = 0}  - (Β_{τ}(s)^{'})^2|_{s = 0} = \frac{6λ^2}{(s + λ)^4}|_{s = 0} - (\frac{2}{λ})^2 = \frac{6}{λ^2} - \frac{4}{λ^2}  = \frac{2}{λ^2}$

In [161]:
# Математическое ожидание времени безотказной работы
MX <- 2/lambda
MX

[1] 6.943004

In [162]:
# Дисперсия времени безотказной работы
DX <- 2/lambda^2
DX

[1] 24.10266